Author: [Lorenzo Zaccagnini](https://www.linkedin.com/in/lorenzo-zaccagnini/)

In [0]:
!apt install libmpc-dev

In [0]:
!pip install gmpy2

In [0]:
import math
import gmpy2
from itertools import chain
from random import randint
import numpy
from datetime import datetime as dt

# Crack RSA
In this notebook I'll try to break an RSA encryption.

This is based on the amazing lessons of Prof. Bill Buchanan, in particular [Cracking Weak RSA with Python](https://www.youtube.com/watch?v=iJuIJLtUVMo)

### What is RSA
RSA (Rivest–Shamir–Adleman) is one of the first public-key cryptosystems and is widely used for secure data transmission. In such a cryptosystem, the encryption key is public and it is different from the decryption key which is kept secret (private). In RSA, this asymmetry is based on the practical difficulty of the factorization of the product of two large prime numbers, the "factoring problem". ([Wikipedia](https://en.wikipedia.org/wiki/RSA_(cryptosystem)))



### Let's define the first parameters

Insert the public exponent **e**

In [1]:
e=int(input())
print ("e is: ", e)

65537
e is:  65537


Insert **n** (the product of two the distinct prime numbers p and q)

In [2]:
n= int(input())
print ("n is: ", n)

551192507892523087002175107206796139
n is:  551192507892523087002175107206796139


Insert the encrypted message **c** (cipher)

In [0]:
c=int(input())
print ("cipher is: ", c)

366309566851082221204562001465846128
cipher is:  366309566851082221204562001465846128


**The public key is (n, e)**

In [5]:
print('Public key \nn: {0} \ne: {1}'
     .format(n, e))

Public key 
n: 551192507892523087002175107206796139 
e: 65537


### Factoring using the Elliptic Curve Method (ECM)
Pyecm factors large integers (up to 50 digits) using the Elliptic Curve Method (ECM), a fast factoring algorithm.

Source [pyecm](https://github.com/martingkelly/pyecm)

Given p and q, **it’s easy to find their product**, n = pq. Instead given such an n, it appears to be **quite hard to recover the prime factors** p and q.

Source [The Mathematics of the RSA Public-Key Cryptosystem](http://www.mathaware.org/mam/06/Kaliski.pdf)

With ECM we'll able to find p and q, **hopefully**.

In [0]:
!wget https://raw.githubusercontent.com/martingkelly/pyecm/master/pyecm.py

--2020-01-25 09:31:39--  https://raw.githubusercontent.com/martingkelly/pyecm/master/pyecm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41473 (41K) [text/plain]
Saving to: ‘pyecm.py’

pyecm.py            100%[===================>]  40.50K  --.-KB/s    in 0.01s   

2020-01-25 09:31:39 (3.24 MB/s) - ‘pyecm.py’ saved [41473/41473]



In [0]:
import pyecm

In [0]:
p, q = list(pyecm.factors(n, False, True,10 ,1))

print('p: {0} \nq: {1}'
     .format(p, q))

p: 892710084053067989 
q: 617437304382188351


### Find the decryption key d
Given n, e, c, and the prime factors p and q, **it’s easy to**
recover the value m. Given only n, e, and c, but not the prime factors, **it appears to be quite hard** to recover the value m. 

Time for some math, calculate 

$Φ = (p−1)*(q−1) $ 

In [0]:
PHI = (p-1)*(q-1)

After we get Φ, let's find the decryption key d using this formula: 

$d=e^{-1}$ mod $Φ$

In [0]:
d=(gmpy2.invert(e, PHI))

### Decrypt the cipher
The formula in order to decrypt it is: $message=q^d$ mod $n$

In [0]:
message = pow(c, d, n)

print ('the decrypted message is: ', message)

the decrypted message is:  1855


**Test**

In [0]:
if c == pow(message, e, n):
  print ('RSA cracked!')
else:
  print ('Epic Fail!')

RSA cracked!


### Further possible improvements



*   Faster computation with GPU and CUDA
*   Schor algorithm on a Quantum computer :)

